# Análisis completo combinando filtrado, agrupación y merge

## Importar bibliotecas

In [1]:
import pandas as pd
import numpy as np

print('Versiones:')
print(f'  - pandas: {pd.__version__}')
print(f'  - numpy:  {np.__version__}')

Versiones:
  - pandas: 2.3.3
  - numpy:  2.3.4


## Crear datasets relacionados

In [2]:
# Dataset de ventas
ventas = pd.DataFrame({
    'id_venta': range(1, 11),
    'id_cliente': np.random.choice([1, 2, 3, 4, 5], 10),
    'id_producto': np.random.choice([101, 102, 103, 104], 10),
    'cantidad': np.random.randint(1, 5, 10),
    'fecha': pd.date_range('2024-01-01', periods=10, freq='D')
})

# Dataset de clientes
clientes = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4, 5],
    'nombre': ['Ana', 'Carlos', 'María', 'Juan', 'Luis'],
    'ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Sevilla']
})

# Dataset de productos
productos = pd.DataFrame({
    'id_producto': [101, 102, 103, 104],
    'nombre': ['Laptop', 'Mouse', 'Teclado', 'Monitor'],
    'precio': [1200, 25, 80, 300],
    'categoria': ['Electrónica', 'Accesorios', 'Accesorios', 'Electrónica']
})

print("Datasets creados:")
print(f"Ventas: {ventas.shape}")
print(f"Clientes: {clientes.shape}")
print(f"Productos: {productos.shape}")

Datasets creados:
Ventas: (10, 5)
Clientes: (5, 3)
Productos: (4, 4)


In [3]:
print(f"Ventas:\n{ventas}\n")
print(f"Clientes:\n{clientes}\n")
print(f"Productos:\n{productos}\n")

Ventas:
   id_venta  id_cliente  id_producto  cantidad      fecha
0         1           1          102         4 2024-01-01
1         2           4          101         4 2024-01-02
2         3           1          104         3 2024-01-03
3         4           4          101         4 2024-01-04
4         5           4          103         3 2024-01-05
5         6           2          101         3 2024-01-06
6         7           2          104         1 2024-01-07
7         8           2          101         3 2024-01-08
8         9           1          101         4 2024-01-09
9        10           2          104         4 2024-01-10

Clientes:
   id_cliente  nombre     ciudad
0           1     Ana     Madrid
1           2  Carlos  Barcelona
2           3   María     Madrid
3           4    Juan   Valencia
4           5    Luis    Sevilla

Productos:
   id_producto   nombre  precio    categoria
0          101   Laptop    1200  Electrónica
1          102    Mouse      25   Accesorio

## Filtrado avanzado con query()

In [4]:
# Ventas del mes actual con query
ventas_recientes = ventas.query('fecha >= "2024-01-05"')
print(f"\nVentas recientes: {len(ventas_recientes)}")

# Productos caros usando variable externa
precio_limite = 100
productos_caros = productos.query('precio >= @precio_limite')
print(f"Productos caros (>= {precio_limite}): {productos_caros['nombre'].tolist()}")


Ventas recientes: 6
Productos caros (>= 100): ['Laptop', 'Monitor']


## Agrupación y agregación

In [5]:
# Ventas por producto
ventas_por_producto = ventas.groupby('id_producto')['cantidad'].sum()
print(f"\nVentas por producto:\n{ventas_por_producto}")

# Estadísticas por cliente
stats_por_cliente = ventas.groupby('id_cliente').agg({
    'cantidad': ['sum', 'mean'],
    'id_venta': 'count'
})
print(f"\nEstadísticas por cliente:\n{stats_por_cliente}")


Ventas por producto:
id_producto
101    18
102     4
103     3
104     8
Name: cantidad, dtype: int32

Estadísticas por cliente:
           cantidad           id_venta
                sum      mean    count
id_cliente                            
1                11  3.666667        3
2                11  2.750000        4
4                11  3.666667        3


## Merge para análisis completo

In [6]:
# Unir ventas con productos
ventas_productos = pd.merge(ventas, productos, on='id_producto')

# Calcular totales
ventas_productos['total'] = ventas_productos['cantidad'] * ventas_productos['precio']

# Unir con clientes
analisis_completo = pd.merge(ventas_productos, clientes, on='id_cliente')

# modificación de nombres de columnas
analisis_completo.rename(columns={'nombre_x':'producto', 'nombre_y':'cliente'}, inplace=True)

print(f"\nAnálisis completo (primeras 5 filas):\n{analisis_completo.head()}")

# Análisis por ciudad
ventas_por_ciudad = analisis_completo.groupby('ciudad')['total'].sum()
print(f"\nVentas totales por ciudad:\n{ventas_por_ciudad}")


Análisis completo (primeras 5 filas):
   id_venta  id_cliente  id_producto  cantidad      fecha producto  precio  \
0         1           1          102         4 2024-01-01    Mouse      25   
1         2           4          101         4 2024-01-02   Laptop    1200   
2         3           1          104         3 2024-01-03  Monitor     300   
3         4           4          101         4 2024-01-04   Laptop    1200   
4         5           4          103         3 2024-01-05  Teclado      80   

     categoria  total cliente    ciudad  
0   Accesorios    100     Ana    Madrid  
1  Electrónica   4800    Juan  Valencia  
2  Electrónica    900     Ana    Madrid  
3  Electrónica   4800    Juan  Valencia  
4   Accesorios    240    Juan  Valencia  

Ventas totales por ciudad:
ciudad
Barcelona    8700
Madrid       5800
Valencia     9840
Name: total, dtype: int64


## Filtrado final

In [7]:
# Clientes con compras > 1000
clientes_top = analisis_completo.groupby(['id_cliente', 'cliente'])['total'].sum()
clientes_top = clientes_top[clientes_top > 1000]
print(f"\nClientes con compras > 1000:\n{clientes_top}")


Clientes con compras > 1000:
id_cliente  cliente
1           Ana        5800
2           Carlos     8700
4           Juan       9840
Name: total, dtype: int64
